# Light sources

In [ ]:
import torch
import torch.nn as nn
import torchlensmaker as tlm
import torch.optim as optim

# Idea: Renderwing of light sources: render a bit into the negative t with different color?
# + render the source 'outline' as a line/disk/surface

In [ ]:
# RaySource
optics = tlm.Sequential(
    tlm.Turn([20, 0]),
    tlm.RaySource(material="air")
)

tlm.show(optics, dim=2, end=40, sampling={})
tlm.show(optics, dim=3, end=40, sampling={})

In [ ]:
# PointSourceAtInfinity
optics = nn.Sequential(
    tlm.Gap(10),
    tlm.Rotate(
        tlm.PointSourceAtInfinity(beam_diameter=18.5),
        angles = (-15, -5),
    ),
    tlm.Gap(10),
    tlm.PointSourceAtInfinity(beam_diameter=18.5),
)

tlm.show(optics, dim=2, end=40, sampling={"base": 30, "sampler": "random"}, color_dim="base")
tlm.show(optics, dim=3, end=40, sampling={"base": 50, "sampler": "uniform"}, color_dim="base")

In [ ]:
optics = nn.Sequential(
    tlm.Gap(-10),
    tlm.PointSource(10),
)

tlm.show(optics, dim=2, end=30, sampling={"base": 10, "sampler": "random"}, color_dim="base")
tlm.show(optics, dim=3, end=100, sampling={"base": 100, "sampler": "random"}, color_dim="base")

In [ ]:
surface = tlm.surfaces.Parabola(diameter=15, a=tlm.parameter(0.02))
lens = tlm.BiLens(surface, material = 'BK7-nd', outer_thickness=1.0)

optics = nn.Sequential(
    tlm.ObjectAtInfinity(beam_diameter=10, angular_size=25),
    tlm.Gap(20),
    lens,
)

tlm.show(optics, dim=2, end=50, color_dim="object")
tlm.show(optics, dim=3, end=200, color_dim="object", sampling={"base": 20, "object": 20, "sampler": "uniform"})

In [ ]:
surface = tlm.surfaces.Parabola(diameter=15, a=tlm.parameter(0.02))
lens = tlm.BiLens(surface, material = 'BK7-nd', outer_thickness=1.0)

object_distance = 50

optics = nn.Sequential(
    tlm.Gap(-object_distance),
    tlm.Object(beam_angular_size=5, object_diameter=5),
    tlm.Gap(object_distance),
    tlm.Gap(20),
    lens,
)

tlm.show(optics, dim=2, end=200, color_dim="object")
tlm.show(optics, dim=3, end=200, sampling={"base": 10, "object": 10}, color_dim="object")

## tlm.Wavelength

Rays can be given a wavelength in different ways:

* **Achromatic rays**. Don't add any `Wavelength` object after the light source. The simulation will be achromatic. In an achromatic system, refractive surfaces with a dispersive material will raise an error. You can still use refractive surfaces but only with a non dispersive material model.

* **Monochromatic rays**. Add a `Wavelength` object with a single element wavelength list, for example: `Wavelength([650])`. This will assign 650nm to all rays. It does not increase the number of rays when sampling.

* **Fixed list of wavelengths**. Add a `Wavelength` object with a list of wavelengths, for example: `Wavelength([450, 500, 550])`. This duplicates existing non chromatic rays to make one ray per wavelength. When sampling, this multiplies the number of rays by the length of the list.

* **Sampling of wavelengths**. Add a `Wavelength` object with two float arguments indicating the range of wavelengths to sample, for example: `Wavelength(400, 800)`. This generates N samples in chromatic space (following the sampling configuration) and duplicates rays to make one ray per sampled wavelength. When sampling, this multiplies the number of rays by the "wavelength" sampling value. As with every other sampling dimension, any sampler can be used (linear, random, normal...)

In [ ]:
# TODO SingleRay light source (no base coordinate)

# Monochromatic
# Multichromatic
# ChromaticRange

optics = nn.Sequential(
    tlm.PointSource(beam_angular_size=5),
    #tlm.Monochromatic(600),
    tlm.Multichromatic([600, 650, 700]),
    #tlm.Chromatic([600, 650, 700]),
    #tlm.Wavelength(600, 700)

    tlm.Gap(10),
    tlm.RefractiveSurface(tlm.Sphere(30, 60), material="SF10"),
)

output = optics(tlm.default_input(dim=2, dtype=torch.float64, sampling={"base": 10}))

tlm.show(optics, dim=2, end=50, color_dim="wavelength")